In [11]:
from tensorflow.keras.preprocessing import text
# from tensorflow.keras.utils import np_utils
from tensorflow.keras.preprocessing import sequence
from nltk.corpus import gutenberg
from string import punctuation
import numpy as np
import re
import nltk

## Step 1: Building the corpus vocabulary

In [12]:
wpt = nltk.WordPunctTokenizer()
stop_words = nltk.corpus.stopwords.words('english')
def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    # tokenize document
    tokens = wpt.tokenize(doc)
    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

normalize_corpus = np.vectorize(normalize_document)

In [69]:
# Loading the bible corpus
bible = gutenberg.sents('bible-kjv.txt')
remove_terms  = punctuation + '0123456789' # all the punctuations plus digits will be removed

norm_bible = [[word.lower() for word in sent if word not in remove_terms] for sent in bible]
# converting each sent from list of strings to strings
norm_bible = [' '.join(tok_sent) for tok_sent in norm_bible] 
# Filter empty sentences
norm_bible = filter(None,normalize_corpus(norm_bible))
# Keep only sentences with more than 2 words
norm_bible = [tok_sent for tok_sent in norm_bible if len(tok_sent.split()) > 2]

In [71]:
print('Total lines:', len(bible))
print('\nSample line:', bible[10])
print('\nProcessed line:', norm_bible[10])

Total lines: 30103

Sample line: ['1', ':', '6', 'And', 'God', 'said', ',', 'Let', 'there', 'be', 'a', 'firmament', 'in', 'the', 'midst', 'of', 'the', 'waters', ',', 'and', 'let', 'it', 'divide', 'the', 'waters', 'from', 'the', 'waters', '.']

Processed line: god said let firmament midst waters let divide waters waters


In [11]:
tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(norm_bible)